In [204]:
from bs4 import BeautifulSoup
import requests
import time

In [40]:
headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
html_page = requests.get('https://www.trustpilot.com/review/hellofresh.com', headers = headers)
soup = BeautifulSoup(html_page.content, 'html.parser')

In [288]:
s = soup.prettify

In [76]:
content = soup.find('div', class_ = "styles_reviewContent__0Q2Tg")

In [77]:
content.find('p').text

"Horrible company to deal, impossible to cancel an order that hasn't even happened yet. They charge your card even if you do manage to cancel your order. No reminders at all about upcoming meals in case you wanted to cancel or suspend deliveries. Customer service on the phone is horrendous all they say is I can't do that, they'll never grab a manager to help fix the problem. Horrible Horrible company"

In [289]:
new = soup.findAll('div', class_ = "styles_reviewCardInner__EwDq2")

In [290]:
text = soup.findAll('p', class_ = 'typography_body-l__KUYFJ')

In [236]:
def scrape_review_pages(page_start, page_end):
    headers = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    reviews = []
    for i in range(page_start, page_end):
        url = 'https://www.trustpilot.com/review/hellofresh.com?page=' + str(i)
        try:
            html_page = requests.get(url, headers = headers)
            #print(url)
            html_page.raise_for_status()
            soup = BeautifulSoup(html_page.content, 'html.parser')
            review_elements = soup.find_all("article", class_="styles_reviewCard__hcAvl")
            for review_element in review_elements:
                review_text = review_element.find("p", class_="typography_body-l__KUYFJ")
                #print(review_text)
                if review_text is not None:
                    review_text = review_text.text.strip()
                else:
                    review_text = "No review text available."
                review_score_tag = review_element.find("div", class_="star-rating_starRating__4rrcf")
                if review_score_tag is not None:
                    review_score = int(review_score_tag.img['alt'].split()[1])
                else:
                    review_score = 0
                #print(review_score)
                review_object = {"text": review_text, "score": review_score}
                reviews.append(review_object)
            time.sleep(3)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {i}: {e}")
            continue
    return reviews

In [222]:
Hello_fresh_reviews = scrape_review_pages(2,275)
hf2 = scrape_review_pages(275,447)
hf3 = scrape_review_pages(447, 597)
hf4 = scrape_review_pages(597, 747)
hf5 = scrape_review_pages(747,900)
hf6 = scrape_review_pages(900,1100)
hf7 = scrape_review_pages(1100,1275)
hf8 = scrape_review_pages(1275,1475)
hf9 = scrape_review_pages(1475,1675)
hf10 = scrape_review_pages(1675,1875)
hf11 = scrape_review_pages(1875,2075)
hf12 = scrape_review_pages(2075,2200)
hf13 = scrape_review_pages(2200,2346)

In [271]:
import pandas as pd
df = pd.DataFrame(Hello_fresh_reviews)
df2 = pd.DataFrame(hf2)
df3 = pd.DataFrame(hf3)
df4 = pd.DataFrame(hf4)
df5 = pd.DataFrame(hf5)
df6 = pd.DataFrame(hf6)
df7 = pd.DataFrame(hf7)
df8 = pd.DataFrame(hf8)
df9 = pd.DataFrame(hf9)
df10 = pd.DataFrame(hf10)
df11 = pd.DataFrame(hf11)
df12 = pd.DataFrame(hf12)
df13 = pd.DataFrame(hf13)

In [313]:
fresh_df = pd.concat([df, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13], 0)

In [314]:
hello_fresh_df = fresh_df.reset_index(drop = True)

In [316]:
hello_fresh_df[hello_fresh_df['text'] == "No review text available."].index

Int64Index([  121,   131,   135,   141,   149,   151,   154,   158,   163,
              165,
            ...
            42940, 42954, 42997, 43191, 43348, 44226, 45000, 45255, 45315,
            45731],
           dtype='int64', length=2352)

In [317]:
hello_fresh_df = hello_fresh_df.drop(hello_fresh_df[hello_fresh_df['text'] == "No review text available."].index)

In [318]:
hello_fresh_df = hello_fresh_df.reset_index(drop=True)

In [320]:
changing_numeric_tosentiment = {1 : "negative", 
                              2:"negative", 
                              3: "neutral", 
                              4: "positive", 
                              5: "positive" }
hello_fresh_df['score'] = hello_fresh_df['score'].replace(changing_numeric_tosentiment)

In [323]:
hello_fresh_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44528 entries, 0 to 44527
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44528 non-null  object
 1   score   44528 non-null  object
dtypes: object(2)
memory usage: 695.9+ KB


## Topic Modeling

In [324]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re  
import nltk 
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag

In [331]:
def process_text(text, min_length):
    
    # get common stop words that we'll remove during tokenization/text normalization
    stop_words = stopwords.words('english')

    #initialize lemmatizer
    wnl = WordNetLemmatizer()

    # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None
   

    # lower case everything
    text_lower = text.lower()

    #remove mentions, hashtags, and urls, strip whitspace and breaks
    text_lower = re.sub(r"@[a-z0-9_]+|#[a-z0-9_]+|http\S+", "", text_lower).strip().replace("\r", "").replace("\n", "").replace("\t", "")
    
    
    # remove stop words and punctuations 
    text_norm = [x for x in word_tokenize(text_lower) if ((x.isalpha()) & (x not in stop_words)) ]

    #  POS detection on the result will be important in telling Wordnet's lemmatizer how to lemmatize
    
    # creates list of tuples with tokens and POS tags in wordnet format
    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(text_norm))) 

    # now we are going to have a cutoff here. any tokenized cocument with length < min length will be removed from corpus
    if len(wordnet_tagged) <= min_length:
        return ''
    else:
         # rejoins lemmatized sentence 
        text_norm = " ".join([wnl.lemmatize(x[0], x[1]) for x in wordnet_tagged if x[1] is not None])
        return text_norm

In [335]:
hello_fresh_df['text'] = hello_fresh_df['text'].apply(process_text, args = [5])

In [343]:
hello_fresh_new = hello_fresh_df[hello_fresh_df['text'] != '']

In [345]:
hello_fresh_new = hello_fresh_new.reset_index(drop = True)

In [346]:
corpus = hello_fresh_new['text']

In [348]:
corpus

0        almost everything include make meal home peopl...
1        recipe taste good food fresh direction clear h...
2        try many meal kit company hello fresh best imp...
3        delivery miss altogether meal ingredient deliv...
4        food fresh portion nicely many option choose r...
                               ...                        
40804    hellofresh treat horribly sign meal people del...
40805    sure try thought find coupon try well get box ...
40806    bore detail get bad company ever deal entire m...
40807    try hello fresh month delivery miss ingredient...
40808    second time rating company request original re...
Name: text, Length: 40809, dtype: object